# Agent Simulation Demo

This notebook demonstrates the agentic workflow system for automated flip opportunity detection.

## System Overview

The agent system consists of 6 specialized agents:

1. **DataRefreshAgent** - Monitors for new Zillow data releases
2. **ScoringAgent** - Computes opportunity scores using the scoring engine
3. **OpportunityDetectionAgent** - Identifies new and changed opportunities
4. **PropertyAnalysisAgent** - Performs deep-dive analysis on top opportunities
5. **AlertAgent** - Generates and manages alerts with priority classification
6. **ReportGeneratorAgent** - Creates weekly summary reports

In [ ]:
import sys
from pathlib import Path
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

from src.data_loader import load_all_datasets
from src.scoring_engine import flip_opportunity_score, BALANCED
from src.agent_workflow import AgentOrchestrator, AgentState
from src.alert_system import AlertManager, AlertType
from src.property_analyzer import PropertyAnalyzer

print("Imports successful!")

## 1. Load Simulation Results

If you've run the simulation (`python workflows/simulate_agent_run.py`), we can load and analyze the results.

In [ ]:
# Load simulation results
agent_logs_dir = Path.cwd().parent / "data" / "processed" / "agent_logs"

# Check if simulation has been run
if not agent_logs_dir.exists():
    print("Simulation not yet run. Execute: python workflows/simulate_agent_run.py")
else:
    # Load summary
    with open(agent_logs_dir / "simulation_summary.json", 'r') as f:
        summary = json.load(f)
    
    print("=" * 50)
    print("SIMULATION SUMMARY")
    print("=" * 50)
    for key, value in summary.items():
        print(f"{key}: {value}")

## 2. Analyze Alert Distribution

In [ ]:
# Load alerts
with open(agent_logs_dir / "alerts.json", 'r') as f:
    alerts = json.load(f)

alerts_df = pd.DataFrame(alerts)
print(f"Total alerts loaded: {len(alerts_df)}")

# Alert priority distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart of priorities
priority_counts = alerts_df['priority'].value_counts()
colors = {'HOT': '#FF4444', 'WARM': '#FFA500', 'WATCH': '#FFFF00', 'INFO': '#0088FF'}
pie_colors = [colors.get(p, '#808080') for p in priority_counts.index]

axes[0].pie(priority_counts.values, labels=priority_counts.index, autopct='%1.1f%%', 
            colors=pie_colors, startangle=90)
axes[0].set_title('Alert Priority Distribution')

# Bar chart by state
top_states = alerts_df['state'].value_counts().head(10)
sns.barplot(x=top_states.values, y=top_states.index, ax=axes[1], palette='viridis')
axes[1].set_xlabel('Number of Alerts')
axes[1].set_ylabel('State')
axes[1].set_title('Top 10 States by Alert Count')

plt.tight_layout()
plt.show()

## 3. Timeline Analysis

In [ ]:
# Load timeline data
timeline = pd.read_csv(agent_logs_dir / "timeline_data.csv")
timeline['date'] = pd.to_datetime(timeline['date'])

print(f"Timeline spans {len(timeline)} days")
print(f"Date range: {timeline['date'].min()} to {timeline['date'].max()}")

# Plot cumulative metrics
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Cumulative opportunities and alerts
axes[0].plot(timeline['date'], timeline['cumulative_opportunities'], 
             label='Cumulative Opportunities', color='steelblue', linewidth=2)
axes[0].plot(timeline['date'], timeline['cumulative_alerts'], 
             label='Cumulative Alerts', color='coral', linewidth=2)
axes[0].set_ylabel('Count')
axes[0].set_title('Cumulative Opportunities and Alerts Over Time')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Daily activity
axes[1].bar(timeline['date'], timeline['new_opportunities'], 
            label='New Opportunities', color='steelblue', alpha=0.7)
axes[1].bar(timeline['date'], timeline['alerts_generated'], 
            label='Alerts Generated', color='coral', alpha=0.7, bottom=timeline['new_opportunities'])
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Count')
axes[1].set_title('Daily Agent Activity')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. HOT Alert Analysis

In [ ]:
# Analyze HOT alerts
hot_alerts = alerts_df[alerts_df['priority'] == 'HOT'].copy()
print(f"Total HOT alerts: {len(hot_alerts)}")

# Display top HOT alerts by score
top_hot = hot_alerts.nlargest(10, 'current_score')[[
    'zip_code', 'city', 'state', 'metro', 'current_score', 'current_value'
]].copy()

top_hot['current_value'] = top_hot['current_value'].apply(lambda x: f"${x:,.0f}")
top_hot['current_score'] = top_hot['current_score'].round(1)

print("\nTop 10 HOT Alerts by Score:")
display(top_hot)

In [ ]:
# Score distribution of HOT alerts
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Score histogram
axes[0].hist(hot_alerts['current_score'], bins=20, color='#FF4444', edgecolor='white', alpha=0.8)
axes[0].axvline(hot_alerts['current_score'].mean(), color='black', linestyle='--', 
                label=f'Mean: {hot_alerts["current_score"].mean():.1f}')
axes[0].set_xlabel('Score')
axes[0].set_ylabel('Count')
axes[0].set_title('HOT Alert Score Distribution')
axes[0].legend()

# Value distribution
axes[1].hist(hot_alerts['current_value'] / 1000, bins=20, color='#4CAF50', edgecolor='white', alpha=0.8)
axes[1].set_xlabel('Home Value ($K)')
axes[1].set_ylabel('Count')
axes[1].set_title('HOT Alert Home Value Distribution')

plt.tight_layout()
plt.show()

## 5. Property Deep Dive Demo

Use the PropertyAnalyzer to perform a comprehensive analysis on a specific ZIP code.

In [ ]:
# Load datasets for analysis
datasets = load_all_datasets()
print("Datasets loaded")

# Get a top HOT alert ZIP for analysis
sample_zip = hot_alerts.nlargest(1, 'current_score')['zip_code'].iloc[0]
print(f"\nAnalyzing ZIP: {sample_zip}")

In [ ]:
# Initialize analyzer
analyzer = PropertyAnalyzer(datasets)

# Get comprehensive analysis
report = analyzer.analyze_zip(sample_zip)

if report:
    print("=" * 60)
    print(f"PROPERTY ANALYSIS REPORT: {report.zip_code}")
    print(f"Location: {report.city}, {report.state}")
    print(f"Metro: {report.metro}")
    print("=" * 60)
    
    print("\n--- TREND ANALYSIS ---")
    trend = report.trend_analysis
    print(f"Current Value: ${trend.current_value:,.0f}")
    print(f"1-Year Ago Value: ${trend.value_1yr_ago:,.0f}")
    print(f"YoY Change: {trend.yoy_change_pct:.1f}%")
    print(f"Trend: {trend.trend_direction} ({trend.trend_strength})")
    print(f"Volatility Score: {trend.volatility_score:.1f}")
    print(f"Seasonality Detected: {trend.seasonality_detected}")
    
    print("\n--- MOMENTUM ---")
    mom = report.momentum
    print(f"Momentum Score: {mom.momentum_score:.1f}")
    print(f"Momentum Grade: {mom.momentum_grade}")
    print(f"Velocity Score: {mom.velocity_score:.1f}")
    print(f"Demand Score: {mom.demand_score:.1f}")
    
    print("\n--- RISK ASSESSMENT ---")
    risk = report.risk
    print(f"Risk Grade: {risk.risk_grade}")
    print(f"Overall Risk Score: {risk.overall_risk_score:.1f}")
    print(f"Risk Factors: {', '.join(risk.risk_factors)}")
    
    print("\n--- INVESTMENT RECOMMENDATION ---")
    rec = report.recommendation
    print(f"Action: {rec.action}")
    print(f"Confidence: {rec.confidence}")
    print(f"Target Purchase Price: ${rec.target_purchase_price:,.0f}")
    print(f"Estimated ARV: ${rec.estimated_arv:,.0f}")
    print(f"Estimated Profit: ${rec.estimated_profit:,.0f}")
    print(f"Profit Margin: {rec.profit_margin_pct:.1f}%")
    print(f"Hold Period: {rec.recommended_hold_period}")
    print(f"Exit Strategy: {rec.exit_strategy}")
else:
    print(f"Could not analyze ZIP {sample_zip}")

## 6. Agent Log Analysis

In [ ]:
# Load and analyze agent logs
agent_names = [
    'DataRefreshAgent',
    'ScoringAgent',
    'OpportunityDetectionAgent',
    'PropertyAnalysisAgent',
    'AlertAgent',
    'ReportGeneratorAgent'
]

print("Agent Activity Summary:")
print("-" * 50)

for agent_name in agent_names:
    log_file = agent_logs_dir / f"{agent_name}_logs.json"
    if log_file.exists():
        with open(log_file, 'r') as f:
            logs = json.load(f)
        
        if logs:
            completed = len([l for l in logs if l.get('status') == 'completed'])
            print(f"{agent_name}: {len(logs)} total runs, {completed} completed")
        else:
            print(f"{agent_name}: No logs")

## 7. Geographic Opportunity Distribution

In [ ]:
# Analyze geographic distribution of opportunities
geo_summary = alerts_df.groupby('state').agg({
    'zip_code': 'count',
    'current_score': 'mean',
    'current_value': 'mean'
}).rename(columns={
    'zip_code': 'alert_count',
    'current_score': 'avg_score',
    'current_value': 'avg_value'
}).round(1)

geo_summary = geo_summary.sort_values('alert_count', ascending=False)

print("Top 15 States by Alert Count:")
display(geo_summary.head(15))

In [ ]:
# Metro area analysis
metro_summary = alerts_df.groupby('metro').agg({
    'zip_code': 'count',
    'current_score': 'mean',
    'current_value': 'mean'
}).rename(columns={
    'zip_code': 'alert_count',
    'current_score': 'avg_score',
    'current_value': 'avg_value'
}).round(1)

top_metros = metro_summary.nlargest(15, 'alert_count')

# Visualize
fig, ax = plt.subplots(figsize=(12, 6))

colors = plt.cm.RdYlGn(top_metros['avg_score'] / 100)
bars = ax.barh(top_metros.index, top_metros['alert_count'], color=colors)

ax.set_xlabel('Number of Alerts')
ax.set_ylabel('Metro Area')
ax.set_title('Top 15 Metro Areas by Alert Count (color = avg score)')

# Add score labels
for i, (idx, row) in enumerate(top_metros.iterrows()):
    ax.text(row['alert_count'] + 0.5, i, f"{row['avg_score']:.0f}", va='center', fontsize=9)

plt.tight_layout()
plt.show()

## 8. Running the Agent System

Here's how you can run the agent system manually:

In [ ]:
# Example: Initialize the orchestrator
from datetime import datetime

# Create output directory
output_dir = Path.cwd().parent / "data" / "processed" / "agent_demo"
output_dir.mkdir(parents=True, exist_ok=True)

# Initialize orchestrator
orchestrator = AgentOrchestrator(output_dir)
print(f"Orchestrator initialized")
print(f"State: {orchestrator.state}")

In [ ]:
# Run a single agent check
scores_df = flip_opportunity_score(
    datasets=datasets,
    strategy=BALANCED,
    min_home_value=50000,
    max_home_value=500000
)

context = {
    'current_date': datetime.now(),
    'state': orchestrator.state,
    'scores_df': scores_df,
    'previous_scores_df': None
}

# Run data refresh check
data_result = orchestrator.data_refresh_agent.run(context)
print(f"Data Refresh Result: {data_result}")

# Run scoring
scoring_result = orchestrator.scoring_agent.run(context)
print(f"Scoring Result: {scoring_result}")

## Summary

This notebook demonstrated:

1. **Simulation Results** - Loading and analyzing 90-day simulation output
2. **Alert Analysis** - Distribution of HOT/WARM/WATCH alerts
3. **Timeline Visualization** - Cumulative and daily activity trends
4. **Property Deep Dive** - Comprehensive ZIP code analysis
5. **Geographic Patterns** - State and metro-level opportunity distribution
6. **Agent Orchestration** - Manual agent execution example

### Next Steps

- Run the Streamlit dashboard: `streamlit run streamlit_app.py`
- Navigate to the **Agent Monitoring** tab for interactive exploration
- Use the **Property Deep Dive** feature to analyze specific ZIPs
- Download alerts and timeline data for further analysis